In [ ]:
import matplotlib as plt
import pandas as pd
import numpy

df = pd.read_csv('../data/industrial-energy-tariffs.csv')

print(df.head())
print(df.shape)




In [ ]:
# subset it to 1000 rows
df = df.sample(n=100).reset_index(drop=True)
# find nan values
# notes_idx = ~df["Notes"].isnull().values
# df_filter = df[notes_idx]

In [ ]:
df


In [ ]:

elec_x = []
elec_y = []
demand_x = []
demand_y = []

# look at each set of tariffs in the dataframa by looking for the marker of the start of each batch: a value in the 'Notes' column, and within each batch sorting by electric and demand 'type' finding the max and min for each
i = 0
while i < len(df):
    if pd.notnull(df.loc[i, 'Notes']):
        elec_max = 0
        elec_min = numpy.inf
        demand_max = 0
        demand_min = numpy.inf
        while pd.isnull(df.loc[i, 'Notes']):
            if df.loc[i, 'type'] == 'electric':
                if df.loc[i, 'charge (imperial)'] > elec_max:
                    elec_max = df.loc[i, 'charge (imperial)']
                if df.loc[i, 'charge (imperial)'] < elec_min:
                    elec_min = df.loc[i, 'charge (imperial)']
            if df.loc[i, 'type'] == 'demand':
                if df.loc[i, 'charge (imperial)'] > demand_max:
                    demand_max = df.loc[i, 'charge (imperial)']
                if df.loc[i, 'charge (imperial)'] < demand_min:
                    demand_min = df.loc[i, 'charge (imperial)']
            
        elec_x.append(elec_min)
        elec_y.append(elec_max / elec_min)
        demand_x.append(demand_min)
        demand_y.append(demand_max / demand_min)
    i += 1

plt.scatter(elec_x, elec_y)
plt.scatter(demand_x, demand_y)